# softmax回归
:label:`sec_softmax`

在 :numref:`sec_linear_regression`中我们介绍了线性回归。
随后，在 :numref:`sec_linear_scratch`中我们从头实现线性回归。
然后，在 :numref:`sec_linear_concise`中我们使用深度学习框架的高级API简洁实现线性回归。

回归可以用于预测*多少*的问题。
比如预测房屋被售出价格，或者棒球队可能获得的胜场数，又或者患者住院的天数。

事实上，我们也对*分类*问题感兴趣：不是问“多少”，而是问“哪一个”：

* 某个电子邮件是否属于垃圾邮件文件夹？
* 某个用户可能*注册*或*不注册*订阅服务？
* 某个图像描绘的是驴、狗、猫、还是鸡？
* 某人接下来最有可能看哪部电影？

通常，机器学习实践者用*分类*这个词来描述两个有微妙差别的问题：
1. 我们只对样本的“硬性”类别感兴趣，即属于哪个类别；
2. 我们希望得到“软性”类别，即得到属于每个类别的概率。
这两者的界限往往很模糊。其中的一个原因是：即使我们只关心硬类别，我们仍然使用软类别的模型。

## 分类问题
:label:`subsec_classification-problem`

我们从一个图像分类问题开始。
假设每次输入是一个$2\times2$的灰度图像。
我们可以用一个标量表示每个像素值，每个图像对应四个特征$x_1, x_2, x_3, x_4$。
此外，假设每个图像属于类别“猫”“鸡”和“狗”中的一个。

接下来，我们要选择如何表示标签。
我们有两个明显的选择：最直接的想法是选择$y \in \{1, 2, 3\}$，
其中整数分别代表$\{\text{狗}, \text{猫}, \text{鸡}\}$。
这是在计算机上存储此类信息的有效方法。
如果类别间有一些自然顺序，
比如说我们试图预测$\{\text{婴儿}, \text{儿童}, \text{青少年}, \text{青年人}, \text{中年人}, \text{老年人}\}$，
那么将这个问题转变为回归问题，并且保留这种格式是有意义的。

但是一般的分类问题并不与类别之间的自然顺序有关。
幸运的是，统计学家很早以前就发明了一种表示分类数据的简单方法：*独热编码*（one-hot encoding）。
独热编码是一个向量，它的分量和类别一样多。
类别对应的分量设置为1，其他所有分量设置为0。
在我们的例子中，标签$y$将是一个三维向量，
其中$(1, 0, 0)$对应于“猫”、$(0, 1, 0)$对应于“鸡”、$(0, 0, 1)$对应于“狗”：

$$y \in \{(1, 0, 0), (0, 1, 0), (0, 0, 1)\}.$$

## 网络架构

为了估计所有可能类别的条件概率，我们需要一个有多个输出的模型，每个类别对应一个输出。
为了解决线性模型的分类问题，我们需要和输出一样多的*仿射函数*（affine function）。
每个输出对应于它自己的仿射函数。
在我们的例子中，由于我们有4个特征和3个可能的输出类别，
我们将需要12个标量来表示权重（带下标的$w$），
3个标量来表示偏置（带下标的$b$）。
下面我们为每个输入计算三个*未规范化的预测*（logit）：$o_1$、$o_2$和$o_3$。

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$

我们可以用神经网络图 :numref:`fig_softmaxreg`来描述这个计算过程。
与线性回归一样，softmax回归也是一个单层神经网络。
由于计算每个输出$o_1$、$o_2$和$o_3$取决于
所有输入$x_1$、$x_2$、$x_3$和$x_4$，
所以softmax回归的输出层也是全连接层。

![softmax回归是一种单层神经网络](../img/softmaxreg.svg)
:label:`fig_softmaxreg`

为了更简洁地表达模型，我们仍然使用线性代数符号。
通过向量形式表达为$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$，
这是一种更适合数学和编写代码的形式。
由此，我们已经将所有权重放到一个$3 \times 4$矩阵中。
对于给定数据样本的特征$\mathbf{x}$，
我们的输出是由权重与输入特征进行矩阵-向量乘法再加上偏置$\mathbf{b}$得到的。

## 全连接层的参数开销
:label:`subsec_parameterization-cost-fc-layers`

正如我们将在后续章节中看到的，在深度学习中，全连接层无处不在。
然而，顾名思义，全连接层是“完全”连接的，可能有很多可学习的参数。
具体来说，对于任何具有$d$个输入和$q$个输出的全连接层，
参数开销为$\mathcal{O}(dq)$，这个数字在实践中可能高得令人望而却步。
幸运的是，将$d$个输入转换为$q$个输出的成本可以减少到$\mathcal{O}(\frac{dq}{n})$，
其中超参数$n$可以由我们灵活指定，以在实际应用中平衡参数节约和模型有效性
 :cite:`Zhang.Tay.Zhang.ea.2021`。

## softmax运算
:label:`subsec_softmax_operation`

现在我们将优化参数以最大化观测数据的概率。
为了得到预测结果，我们将设置一个阈值，如选择具有最大概率的标签。

我们希望模型的输出$\hat{y}_j$可以视为属于类$j$的概率，
然后选择具有最大输出值的类别$\operatorname*{argmax}_j y_j$作为我们的预测。
例如，如果$\hat{y}_1$、$\hat{y}_2$和$\hat{y}_3$分别为0.1、0.8和0.1，
那么我们预测的类别是2，在我们的例子中代表“鸡”。

然而我们能否将未规范化的预测$o$直接视作我们感兴趣的输出呢？
答案是否定的。
因为将线性层的输出直接视为概率时存在一些问题：
一方面，我们没有限制这些输出数字的总和为1。
另一方面，根据输入的不同，它们可以为负值。
这些违反了 :numref:`sec_prob`中所说的概率基本公理。

要将输出视为概率，我们必须保证在任何数据上的输出都是非负的且总和为1。
此外，我们需要一个训练的目标函数，来激励模型精准地估计概率。
例如，
在分类器输出0.5的所有样本中，我们希望这些样本是刚好有一半实际上属于预测的类别。
这个属性叫做*校准*（calibration）。

社会科学家邓肯·卢斯于1959年在*选择模型*（choice model）的理论基础上
发明的*softmax函数*正是这样做的：
softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持
可导的性质。
为了完成这一目标，我们首先对每个未规范化的预测求幂，这样可以确保输出非负。
为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和。如下式：

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$
:eqlabel:`eq_softmax_y_and_o`

这里，对于所有的$j$总有$0 \leq \hat{y}_j \leq 1$。
因此，$\hat{\mathbf{y}}$可以视为一个正确的概率分布。
softmax运算不会改变未规范化的预测$\mathbf{o}$之间的大小次序，只会确定分配给每个类别的概率。
因此，在预测过程中，我们仍然可以用下式来选择最有可能的类别。

$$
\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.
$$

尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。
因此，softmax回归是一个*线性模型*（linear model）。

## 小批量样本的矢量化
:label:`subsec_softmax_vectorization`

为了提高计算效率并且充分利用GPU，我们通常会对小批量样本的数据执行矢量计算。
假设我们读取了一个批量的样本$\mathbf{X}$，
其中特征维度（输入数量）为$d$，批量大小为$n$。
此外，假设我们在输出中有$q$个类别。
那么小批量样本的特征为$\mathbf{X} \in \mathbb{R}^{n \times d}$，
权重为$\mathbf{W} \in \mathbb{R}^{d \times q}$，
偏置为$\mathbf{b} \in \mathbb{R}^{1\times q}$。
softmax回归的矢量计算表达式为：

$$ \begin{aligned} \mathbf{O} &= \mathbf{X} \mathbf{W} + \mathbf{b}, \\ \hat{\mathbf{Y}} & = \mathrm{softmax}(\mathbf{O}). \end{aligned} $$
:eqlabel:`eq_minibatch_softmax_reg`

相对于一次处理一个样本，
小批量样本的矢量化加快了$\mathbf{X}和\mathbf{W}$的矩阵-向量乘法。
由于$\mathbf{X}$中的每一行代表一个数据样本，
那么softmax运算可以*按行*（rowwise）执行：
对于$\mathbf{O}$的每一行，我们先对所有项进行幂运算，然后通过求和对它们进行标准化。
在 :eqref:`eq_minibatch_softmax_reg`中，
$\mathbf{X} \mathbf{W} + \mathbf{b}$的求和会使用广播机制，
小批量的未规范化预测$\mathbf{O}$和输出概率$\hat{\mathbf{Y}}$
都是形状为$n \times q$的矩阵。

## 损失函数

接下来，我们需要一个损失函数来度量预测的效果。
我们将使用最大似然估计，这与在线性回归
（ :numref:`subsec_normal_distribution_and_squared_loss`）
中的方法相同。

### 对数似然

softmax函数给出了一个向量$\hat{\mathbf{y}}$，
我们可以将其视为“对给定任意输入$\mathbf{x}$的每个类的条件概率”。
例如，$\hat{y}_1$=$P(y=\text{猫} \mid \mathbf{x})$。
假设整个数据集$\{\mathbf{X}, \mathbf{Y}\}$具有$n$个样本，
其中索引$i$的样本由特征向量$\mathbf{x}^{(i)}$和独热标签向量$\mathbf{y}^{(i)}$组成。
我们可以将估计值与实际值进行比较：

$$
P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}).
$$

根据最大似然估计，我们最大化$P(\mathbf{Y} \mid \mathbf{X})$，相当于最小化负对数似然：

$$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
= \sum_{i=1}^n l(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}),
$$

其中，对于任何标签$\mathbf{y}$和模型预测$\hat{\mathbf{y}}$，损失函数为：

$$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$
:eqlabel:`eq_l_cross_entropy`

在本节稍后的内容会讲到， :eqref:`eq_l_cross_entropy`中的损失函数
通常被称为*交叉熵损失*（cross-entropy loss）。
由于$\mathbf{y}$是一个长度为$q$的独热编码向量，
所以除了一个项以外的所有项$j$都消失了。
由于所有$\hat{y}_j$都是预测的概率，所以它们的对数永远不会大于$0$。
因此，如果正确地预测实际标签，即如果实际标签$P(\mathbf{y} \mid \mathbf{x})=1$，
则损失函数不能进一步最小化。
注意，这往往是不可能的。
例如，数据集中可能存在标签噪声（比如某些样本可能被误标），
或输入特征没有足够的信息来完美地对每一个样本分类。

### softmax及其导数
:label:`subsec_softmax_and_derivatives`

由于softmax和相关的损失函数很常见，
因此我们需要更好地理解它的计算方式。
将 :eqref:`eq_softmax_y_and_o`代入损失 :eqref:`eq_l_cross_entropy`中。
利用softmax的定义，我们得到：

$$
\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j\\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}
$$

考虑相对于任何未规范化的预测$o_j$的导数，我们得到：

$$
\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.
$$

换句话说，导数是我们softmax模型分配的概率与实际发生的情况（由独热标签向量表示）之间的差异。
从这个意义上讲，这与我们在回归中看到的非常相似，
其中梯度是观测值$y$和估计值$\hat{y}$之间的差异。
这不是巧合，在任何指数族分布模型中
（参见[本书附录中关于数学分布的一节](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/distributions.html)），
对数似然的梯度正是由此得出的。
这使梯度计算在实践中变得容易很多。

### 交叉熵损失

现在让我们考虑整个结果分布的情况，即观察到的不仅仅是一个结果。
对于标签$\mathbf{y}$，我们可以使用与以前相同的表示形式。
唯一的区别是，我们现在用一个概率向量表示，如$(0.1, 0.2, 0.7)$，
而不是仅包含二元项的向量$(0, 0, 1)$。
我们使用 :eqref:`eq_l_cross_entropy`来定义损失$l$，
它是所有标签分布的预期损失值。
此损失称为*交叉熵损失*（cross-entropy loss），它是分类问题最常用的损失之一。
本节我们将通过介绍信息论基础来理解交叉熵损失。
如果想了解更多信息论的细节，请进一步参考
[本书附录中关于信息论的一节](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html)。

## 信息论基础
:label:`subsec_info_theory_basics`

*信息论*（information theory）涉及编码、解码、发送以及尽可能简洁地处理信息或数据。

### 熵

信息论的核心思想是量化数据中的信息内容。
在信息论中，该数值被称为分布$P$的*熵*（entropy）。可以通过以下方程得到：

$$H[P] = \sum_j - P(j) \log P(j).$$
:eqlabel:`eq_softmax_reg_entropy`

信息论的基本定理之一指出，为了对从分布$p$中随机抽取的数据进行编码，
我们至少需要$H[P]$“纳特（nat）”对其进行编码。
“纳特”相当于*比特*（bit），但是对数底为$e$而不是2。因此，一个纳特是$\frac{1}{\log(2)} \approx 1.44$比特。

### 信息量

压缩与预测有什么关系呢？
想象一下，我们有一个要压缩的数据流。
如果我们很容易预测下一个数据，那么这个数据就很容易压缩。
为什么呢？
举一个极端的例子，假如数据流中的每个数据完全相同，这会是一个非常无聊的数据流。
由于它们总是相同的，我们总是知道下一个数据是什么。
所以，为了传递数据流的内容，我们不必传输任何信息。也就是说，“下一个数据是xx”这个事件毫无信息量。

但是，如果我们不能完全预测每一个事件，那么我们有时可能会感到"惊异"。
克劳德·香农决定用信息量$\log \frac{1}{P(j)} = -\log P(j)$来量化这种惊异程度。
在观察一个事件$j$时，并赋予它（主观）概率$P(j)$。
当我们赋予一个事件较低的概率时，我们的惊异会更大，该事件的信息量也就更大。
在 :eqref:`eq_softmax_reg_entropy`中定义的熵，
是当分配的概率真正匹配数据生成过程时的*信息量的期望*。

### 重新审视交叉熵

如果把熵$H(P)$想象为“知道真实概率的人所经历的惊异程度”，那么什么是交叉熵？
交叉熵*从*$P$*到*$Q$，记为$H(P, Q)$。
我们可以把交叉熵想象为“主观概率为$Q$的观察者在看到根据概率$P$生成的数据时的预期惊异”。
当$P=Q$时，交叉熵达到最低。
在这种情况下，从$P$到$Q$的交叉熵是$H(P, P)= H(P)$。

简而言之，我们可以从两方面来考虑交叉熵分类目标：
（i）最大化观测数据的似然；（ii）最小化传达标签所需的惊异。

## 模型预测和评估

在训练softmax回归模型后，给出任何样本特征，我们可以预测每个输出类别的概率。
通常我们使用预测概率最高的类别作为输出类别。
如果预测与实际类别（标签）一致，则预测是正确的。
在接下来的实验中，我们将使用*精度*（accuracy）来评估模型的性能。
精度等于正确预测数与预测总数之间的比率。

## 小结

* softmax运算获取一个向量并将其映射为概率。
* softmax回归适用于分类问题，它使用了softmax运算中输出类别的概率分布。
* 交叉熵是一个衡量两个概率分布之间差异的很好的度量，它测量给定模型编码数据所需的比特数。

## 练习

1. 我们可以更深入地探讨指数族与softmax之间的联系。
    1. 计算softmax交叉熵损失$l(\mathbf{y},\hat{\mathbf{y}})$的二阶导数。
    1. 计算$\mathrm{softmax}(\mathbf{o})$给出的分布方差，并与上面计算的二阶导数匹配。
1. 假设我们有三个类发生的概率相等，即概率向量是$(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$。
    1. 如果我们尝试为它设计二进制代码，有什么问题？
    1. 请设计一个更好的代码。提示：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联合编码$n$个观测值怎么办？
1. softmax是对上面介绍的映射的误称（虽然深度学习领域中很多人都使用这个名字）。真正的softmax被定义为$\mathrm{RealSoftMax}(a, b) = \log (\exp(a) + \exp(b))$。
    1. 证明$\mathrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$。
    1. 证明$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \mathrm{max}(a, b)$成立，前提是$\lambda > 0$。
    1. 证明对于$\lambda \to \infty$，有$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$。
    1. soft-min会是什么样子？
    1. 将其扩展到两个以上的数字。

[Discussions](https://discuss.d2l.ai/t/1785)


1. 我们可以更深入地探讨指数族与softmax之间的联系。
    1. 计算softmax交叉熵损失$l(\mathbf{y},\hat{\mathbf{y}})$的二阶导数。
    1. 计算$\mathrm{softmax}(\mathbf{o})$给出的分布方差，并与上面计算的二阶导数匹配。

我将详细计算softmax交叉熵损失的二阶导数，并与softmax分布的方差进行比较。

## 1. 计算softmax交叉熵损失的二阶导数

首先回顾softmax交叉熵损失函数：
$$l(\mathbf{y}, \hat{\mathbf{y}}) = -\sum_{j=1}^q y_j \log \hat{y}_j$$

其中 $\hat{y}_j = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)}$ 是softmax函数。

### 一阶导数
我们已经知道一阶导数为：
$$\frac{\partial l}{\partial o_j} = \hat{y}_j - y_j$$

### 二阶导数
计算二阶导数需要考虑两种情况：

**情况1：对同一变量求导 ($j = k$)**
$$\frac{\partial^2 l}{\partial o_j^2} = \frac{\partial}{\partial o_j}(\hat{y}_j - y_j) = \frac{\partial \hat{y}_j}{\partial o_j}$$

由于 $\hat{y}_j = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)}$，使用商法则：

$$\frac{\partial \hat{y}_j}{\partial o_j} = \frac{\exp(o_j)\sum_{k=1}^q \exp(o_k) - \exp(o_j)\exp(o_j)}{(\sum_{k=1}^q \exp(o_k))^2} = \hat{y}_j(1 - \hat{y}_j)$$

**情况2：对不同变量求导 ($j \neq k$)**
$$\frac{\partial^2 l}{\partial o_j \partial o_k} = \frac{\partial}{\partial o_k}(\hat{y}_j - y_j) = \frac{\partial \hat{y}_j}{\partial o_k}$$

$$\frac{\partial \hat{y}_j}{\partial o_k} = \frac{-\exp(o_j)\exp(o_k)}{(\sum_{k'=1}^q \exp(o_{k'}))^2} = -\hat{y}_j\hat{y}_k$$

因此，二阶导数的完整形式为：
$$\frac{\partial^2 l}{\partial o_j \partial o_k} = \begin{cases} 
\hat{y}_j(1 - \hat{y}_j) & \text{if } j = k \\
-\hat{y}_j\hat{y}_k & \text{if } j \neq k
\end{cases}$$

## 2. 计算softmax分布的方差

对于softmax分布，这是一个分类分布，其概率质量函数为 $P(X = j) = \hat{y}_j$。

对于分类分布，方差-协方差矩阵的元素为：

**对角元素（方差）：**
$$\mathrm{Var}[X_j] = \hat{y}_j(1 - \hat{y}_j)$$

**非对角元素（协方差）：**
$$\mathrm{Cov}[X_j, X_k] = -\hat{y}_j\hat{y}_k \quad (j \neq k)$$

这与我们计算的二阶导数完全匹配：

$$\mathrm{Hessian}[l] = \begin{bmatrix}
\hat{y}_1(1-\hat{y}_1) & -\hat{y}_1\hat{y}_2 & \cdots & -\hat{y}_1\hat{y}_q \\
-\hat{y}_2\hat{y}_1 & \hat{y}_2(1-\hat{y}_2) & \cdots & -\hat{y}_2\hat{y}_q \\
\vdots & \vdots & \ddots & \vdots \\
-\hat{y}_q\hat{y}_1 & -\hat{y}_q\hat{y}_2 & \cdots & \hat{y}_q(1-\hat{y}_q)
\end{bmatrix}$$

这个结果表明，softmax交叉熵损失的Hessian矩阵（二阶导数矩阵）正好等于softmax分布的协方差矩阵。这反映了指数族分布的一个重要性质：负对数似然函数的二阶导数等于其对应分布的Fisher信息矩阵，而Fisher信息矩阵又与分布的协方差结构密切相关。

我有个疑问，为什么 求二价导数会分j=k和j!=k的情况，这有什么一样吗？并且这个与方差-协方差矩阵有什么关系。

这是一个很好的问题。让我详细解释为什么需要区分 $j=k$ 和 $j \neq k$ 的情况，以及它与方差-协方差矩阵的关系。

## 为什么区分 j=k 和 j≠k 的情况

当我们计算二阶导数时，我们需要考虑所有可能的偏导数组合。对于向量函数，二阶导数形成一个**海森矩阵(Hessian Matrix)**，其元素为 $\frac{\partial^2 l}{\partial o_j \partial o_k}$。

这个矩阵有两种类型的元素：

1. **对角元素**：当 $j = k$ 时，我们计算的是 $\frac{\partial^2 l}{\partial o_j^2}$（同一个变量的二阶偏导数）
2. **非对角元素**：当 $j \neq k$ 时，我们计算的是 $\frac{\partial^2 l}{\partial o_j \partial o_k}$（不同变量间的混合偏导数）

这两种情况的计算结果是不同的，因此需要分别处理。

## 海森矩阵与方差-协方差矩阵的关系

对于分类分布（Categorical Distribution），其统计特性可以通过方差-协方差矩阵来描述：

- **方差**：$\mathrm{Var}[X_j] = \hat{y}_j(1 - \hat{y}_j)$（单个类别的方差）
- **协方差**：$\mathrm{Cov}[X_j, X_k] = -\hat{y}_j\hat{y}_k$（不同类别间的协方差）

这个结果来自于分类分布的性质：如果一个样本属于类别 $j$，它就不能同时属于类别 $k$（当 $j \neq k$ 时），这种负相关性导致了协方差为负值。

## 具体示例

考虑一个3类分类问题，假设 $\hat{\mathbf{y}} = [0.5, 0.3, 0.2]$，那么：

**海森矩阵（二阶导数矩阵）：**
$$H = \begin{bmatrix}
0.5(1-0.5) & -0.5 \times 0.3 & -0.5 \times 0.2 \\
-0.3 \times 0.5 & 0.3(1-0.3) & -0.3 \times 0.2 \\
-0.2 \times 0.5 & -0.2 \times 0.3 & 0.2(1-0.2)
\end{bmatrix} = \begin{bmatrix}
0.25 & -0.15 & -0.1 \\
-0.15 & 0.21 & -0.06 \\
-0.1 & -0.06 & 0.16
\end{bmatrix}$$

**方差-协方差矩阵：**
$$\Sigma = \begin{bmatrix}
0.25 & -0.15 & -0.1 \\
-0.15 & 0.21 & -0.06 \\
-0.1 & -0.06 & 0.16
\end{bmatrix}$$

两者完全相同！

## 深层含义

这种等价性不是偶然的，它反映了指数族分布的一个重要性质：

1. **Fisher信息矩阵**：对于指数族分布，负对数似然函数的海森矩阵等于该分布的Fisher信息矩阵
2. **协方差关系**：Fisher信息矩阵的逆等于参数估计的协方差矩阵
3. **Cramer-Rao下界**：这为参数估计的精度提供了理论下界

因此，通过这个练习，我们看到：
- softmax交叉熵损失函数的曲率（二阶导数）直接反映了模型预测的不确定性
- 对角元素（方差）表示每个类别预测的不确定性
- 非对角元素（协方差）表示不同类别预测间的相关性

1. 假设我们有三个类发生的概率相等，即概率向量是$(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$。
    1. 如果我们尝试为它设计二进制代码，有什么问题？
    1. 请设计一个更好的代码。提示：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联合编码$n$个观测值怎么办？

让我来分析这个关于信息论编码的问题。

## 1. 为等概率的三类设计二进制代码的问题

当三个类别的概率相等（都是1/3）时，如果试图为它们设计二进制代码，主要问题是：

- **无法实现最优编码**：在二进制系统中，我们只能使用长度为整数的编码（1位、2位等）
- **效率损失**：理论上每个符号所需的最小比特数是 $H(P) = -\sum_{i=1}^{3} \frac{1}{3} \log_2 \frac{1}{3} = \log_2 3 \approx 1.585$ 比特，但实际只能使用2位编码（能表示4个状态），造成编码效率的损失

## 2. 更好的编码方案

### 联合编码多个观测值

正如提示所建议，我们可以对多个独立观测值进行联合编码：

#### 对于两个独立观测：
考虑两个独立的观测值，共有 $3 \times 3 = 9$ 种可能的组合，每种组合的概率都是 $\frac{1}{9}$。

理论上需要的比特数是 $\log_2 9 = 2\log_2 3 \approx 3.17$ 比特，可以用4位二进制编码表示。

#### 对于n个独立观测：
对于n个独立观测，共有 $3^n$ 种可能的组合，每种组合概率为 $\frac{1}{3^n}$。

理论上需要的比特数是 $\log_2(3^n) = n\log_2 3 \approx 1.585n$ 比特。

当n增大时，我们可以设计更接近理论最优值的编码方案。例如：

- 当n=6时，有 $3^6 = 729$ 种组合，需要 $\log_2 729 \approx 9.51$ 比特，可以用10位编码
- 相比之下，单独编码需要 $6 \times 2 = 12$ 比特

这种联合编码方法的优势在于：
1. 更接近信息论下限（熵）
2. 随着n的增加，平均每个符号的编码长度趋近于理论最优值 $\log_2 3$
3. 利用了渐近均分性（Asymptotic Equipartition Property, AEP）

这种方法在实际应用中类似于霍夫曼编码或算术编码的思想，通过考虑更长的符号序列来实现更高的编码效率。

1. softmax是对上面介绍的映射的误称（虽然深度学习领域中很多人都使用这个名字）。真正的softmax被定义为$\mathrm{RealSoftMax}(a, b) = \log (\exp(a) + \exp(b))$。
    1. 证明$\mathrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$。
    1. 证明$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \mathrm{max}(a, b)$成立，前提是$\lambda > 0$。
    1. 证明对于$\lambda \to \infty$，有$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$。
    1. soft-min会是什么样子？
    1. 将其扩展到两个以上的数字。

我来逐一证明这些关于 `RealSoftMax` 函数的性质。

## 1. 证明 $\mathrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$

不失一般性，假设 $a \geq b$，即 $\max(a, b) = a$。

根据 `RealSoftMax` 的定义：
$$\mathrm{RealSoftMax}(a, b) = \log(\exp(a) + \exp(b))$$

我们可以将其重写为：
$$\mathrm{RealSoftMax}(a, b) = \log(\exp(a) + \exp(b)) = \log(\exp(a)(1 + \exp(b-a))) = a + \log(1 + \exp(b-a))$$

由于 $a \geq b$，所以 $b-a \leq 0$，因此 $\exp(b-a) \leq 1$，所以 $1 + \exp(b-a) > 1$。

因为 $\log(1 + \exp(b-a)) > 0$，所以：
$$\mathrm{RealSoftMax}(a, b) = a + \log(1 + \exp(b-a)) > a = \max(a, b)$$

## 2. 证明 $\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \mathrm{max}(a, b)$，其中 $\lambda > 0$

根据定义：
$$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) = \lambda^{-1} \log(\exp(\lambda a) + \exp(\lambda b))$$

同样假设 $a \geq b$，则：
$$\lambda^{-1} \log(\exp(\lambda a) + \exp(\lambda b)) = \lambda^{-1} \log(\exp(\lambda a)(1 + \exp(\lambda(b-a))))$$
$$= \lambda^{-1}(\lambda a + \log(1 + \exp(\lambda(b-a)))) = a + \lambda^{-1}\log(1 + \exp(\lambda(b-a)))$$

由于 $a \geq b$，所以 $b-a \leq 0$。当 $\lambda > 0$ 时，$\lambda(b-a) \leq 0$，因此 $\exp(\lambda(b-a)) \leq 1$。

这意味着 $1 + \exp(\lambda(b-a)) > 1$，所以 $\log(1 + \exp(\lambda(b-a))) > 0$。

因此：
$$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) = a + \lambda^{-1}\log(1 + \exp(\lambda(b-a))) > a = \max(a, b)$$

## 3. 证明当 $\lambda \to \infty$ 时，$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$

还是假设 $a \geq b$。我们有：
$$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) = a + \lambda^{-1}\log(1 + \exp(\lambda(b-a)))$$

当 $\lambda \to \infty$ 时，由于 $b-a \leq 0$:
- 如果 $a > b$，则 $\lambda(b-a) \to -\infty$，所以 $\exp(\lambda(b-a)) \to 0$
- 如果 $a = b$，则 $\lambda(b-a) = 0$，所以 $\exp(\lambda(b-a)) = 1$

在第一种情况下：
$$\lim_{\lambda \to \infty} \lambda^{-1}\log(1 + \exp(\lambda(b-a))) = \lim_{\lambda \to \infty} \lambda^{-1}\log(1 + \exp(\lambda(b-a))) = 0$$

在第二种情况下：
$$\lim_{\lambda \to \infty} \lambda^{-1}\log(1 + \exp(\lambda(b-a))) = \lim_{\lambda \to \infty} \lambda^{-1}\log(2) = 0$$

因此：
$$\lim_{\lambda \to \infty} \lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) = a = \max(a, b)$$

## 4. soft-min 会是什么样子？

基于 `RealSoftMax` 的对偶性，soft-min 可以定义为：
$$\mathrm{RealSoftMin}(a, b) = -\log(\exp(-a) + \exp(-b))$$

验证其性质：
- $\mathrm{RealSoftMin}(a, b) < \min(a, b)$（类似于 `RealSoftMax` 的性质）
- 当 $\lambda \to \infty$ 时，$-\lambda^{-1} \mathrm{RealSoftMin}(-\lambda a, -\lambda b) \to \min(a, b)$

## 5. 扩展到两个以上的数字

对于 $n$ 个数字 $x_1, x_2, ..., x_n$，`RealSoftMax` 可以扩展为：
$$\mathrm{RealSoftMax}(x_1, x_2, ..., x_n) = \log(\exp(x_1) + \exp(x_2) + ... + \exp(x_n))$$

相应的性质也扩展为：
- $\mathrm{RealSoftMax}(x_1, ..., x_n) > \max(x_1, ..., x_n)$
- $\lambda^{-1} \mathrm{RealSoftMax}(\lambda x_1, ..., \lambda x_n) > \max(x_1, ..., x_n)$ 当 $\lambda > 0$
- $\lim_{\lambda \to \infty} \lambda^{-1} \mathrm{RealSoftMax}(\lambda x_1, ..., \lambda x_n) = \max(x_1, ..., x_n)$

这个扩展保持了原始 `RealSoftMax` 的所有关键性质，并且是深度学习中常用的 softmax 函数的对数版本。